In [1]:
import os
import os.path
from tqdm import tqdm
import requests, json
import re
from bs4 import BeautifulSoup
import uuid
import warnings
warnings.filterwarnings("ignore")


In [2]:
BASE_URL = "https://www.myntra.com/"
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
BASE_DIR = './myntra/clothes/'
IMAGE_DIR = "images/"
caption_tags_file = "caption_tags_file.csv"


In [3]:
def download_image(image_url):
    if not os.path.exists(os.path.join(BASE_DIR,IMAGE_DIR)):
        os.makedirs(os.path.join(BASE_DIR,IMAGE_DIR))
        
    img_data = requests.get(image_url).content
    myuuid = uuid.uuid4()
    extension = os.path.splitext(image_url)[1]
    file_name = str(myuuid) + extension    
    image_with_path = os.path.join(BASE_DIR + IMAGE_DIR,file_name)
    with open(image_with_path, 'wb') as handler:
        handler.write(img_data)
    
    return file_name

In [4]:
def get_image_caption(json_data):
    image_caption = ""
    article_details = json_data['pdpData']['articleAttributes']  
    
    sleeve_length = article_details.get('Sleeve Length')    
    if sleeve_length:
        sleeve_length = article_details['Sleeve Length']
        sleeve_length = sleeve_length.replace(' ','')
        image_caption = image_caption + sleeve_length + "|"
    
    neck_style = article_details.get('Neck')
    if neck_style:
        neck_style = article_details['Neck']
        neck_style = neck_style.replace(' ','')
        image_caption = image_caption + neck_style + "|"
    
    sleeve_style = article_details.get('Sleeve Styling')
    if sleeve_style:
        sleeve_style = article_details['Sleeve Styling']
        sleeve_style = sleeve_style.replace(' ','')        
        image_caption = image_caption + sleeve_style
    
    if image_caption[-1] == '|':
        image_caption = image_caption[:(len(image_caption))-1]
    
    
    return image_caption

In [5]:
def extract_url_details(landing_url):
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
    
    s = requests.Session()
    res = s.get(landing_url.strip(), headers=headers, verify=False)
    match = re.findall(r"<script>window.__myx = (.+?)</script>", res.text)
    
    json_data = json.loads(match[0])
    product_title = json_data['pdpData']['productDetails']

    images = json_data['pdpData']['media']
    image_url = images['albums'][0]['images'][0]['src']
    
    image_file_name = download_image(image_url=image_url)       

    image_caption = get_image_caption(json_data=json_data)
    image_caption = "{}, {}\n".format(image_file_name, image_caption)
    
    image_dir = BASE_DIR + "caption_tags_file.csv"
    caption_file_handle = open(image_dir,'a+')
    caption_file_handle.write(image_caption)

In [6]:
def test_fun(url):
    s = requests.Session()
    res = s.get("https://www.myntra.com/tops/paralians/paralians-blue-embroidered-crop-top/18413448/buy", headers=headers, verify=False)
    match = re.findall(r"<script>window.__myx = (.+?)</script>", res.text)

    json_data = json.loads(match[0])
    product_title = json_data['pdpData']['productDetails']

    images = json_data['pdpData']['media']
    image_url = images['albums'][0]['images'][0]['src']
    print(image_url)

In [7]:

url_list_path = BASE_DIR + "full_url_list.csv"
url_list_path_handle = open(url_list_path,'r+')

url_list = []
for url in url_list_path_handle:
    url_list.append(url)

for url in tqdm(url_list[:300],colour="green"):
    extract_url_details(url)


100%|██████████| 300/300 [10:43<00:00,  2.14s/it]
